### Book Classifier: Data Exploration

Dataset found here: http://www.cs.cmu.edu/~dbamman/booksummaries.html
        
This notebook explored the data from above, with an attempt to determine the TYPE of classifier we will use.

* classification type: Binary, multi, knn?



In [3]:
import pandas as pd
import numpy as np

In [51]:

cols = ['wiki_id', 'freebase_id', 'title','author', 'pub_data', 'genre','summary']
df = pd.read_csv('../data/raw/booksummaries/booksummaries.tsv',  sep='\t', names=cols)

df[:5]

,wiki_id,freebase_id,title,author,pub_data,genre,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [23]:
### are the categories the same?

genre = df['genre']
print(genre[0])
print(genre[1])
print(genre[2])
print(genre[3])
print(genre[4])

df.dtypes


{"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children's literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}
{"/m/06n90": "Science Fiction", "/m/0l67h": "Novella", "/m/014dfn": "Speculative fiction", "/m/0c082": "Utopian and dystopian fiction", "/m/06nbt": "Satire", "/m/02xlf": "Fiction"}
{"/m/02m4t": "Existentialism", "/m/02xlf": "Fiction", "/m/0pym5": "Absurdist fiction", "/m/05hgj": "Novel"}
nan
{"/m/03lrw": "Hard science fiction", "/m/06n90": "Science Fiction", "/m/014dfn": "Speculative fiction", "/m/01hmnh": "Fantasy", "/m/02xlf": "Fiction"}


wiki_id         int64
freebase_id    object
title          object
author         object
pub_data       object
genre          object
summary        object
dtype: object

In [44]:
## can we load this column as a json value?
import json
from pprint import pprint
j = json.loads('{"/m/02m4t": "Existentialism", "/m/02xlf": "Fiction", "/m/0pym5": "Absurdist fiction", "/m/05hgj": "Novel"}')
pprint(j)

{'/m/02m4t': 'Existentialism',
 '/m/02xlf': 'Fiction',
 '/m/05hgj': 'Novel',
 '/m/0pym5': 'Absurdist fiction'}


In [70]:
## cool, so let's add a column to the df with a list of genre's, no need for the weird number that is the key

df = df[df['genre'].notnull()]

def make_list(col):
    j = json.loads(col)
    new_list = []
    for k, v in j.items():
        new_list.append(v)
        
    return new_list

df['new_genres'] = df.apply(lambda row: make_list(row['genre']), axis=1)

df[:5]




,wiki_id,freebase_id,title,author,pub_data,genre,summary,new_genres
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N..."
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul..."
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge...","[War novel, Roman à clef]"


In [64]:
## let's get a count of all the distinct genres

## explode is a cool function 

v = df['new_genres'].explode()
v

0                 Roman à clef
0                       Satire
0        Children's literature
0          Speculative fiction
0                      Fiction
                 ...          
16555                 Thriller
16555                  Fiction
16556            Autobiography
16558         Epistolary novel
16558      Speculative fiction
Name: new_genres, Length: 30004, dtype: object

In [67]:
### lets do some filtering here
x = v.value_counts()

## remove the basic categories




Fiction                   4747
Speculative fiction       4314
Science Fiction           2870
Novel                     2463
Fantasy                   2413
                          ... 
Superhero fiction            1
Fantastique                  1
Pastiche                     1
Medieval romance             1
First-person narrative       1
Name: new_genres, Length: 227, dtype: int64

In [71]:
## science fiction seems like a good category for us to build our first binary classifier:

## first let's get a dataframe of all the text with science fiction in the new_genre's list

## get text with Science Fiction label


def set_category(genres, cat):
    for item in genres:
        if item == cat:
            return 1
    return 0

df['Science Fiction']= df.apply(lambda row: set_category(row['new_genres'], 'Science Fiction'), axis=1)
df[:5]

,wiki_id,freebase_id,title,author,pub_data,genre,summary,new_genres,Science Fiction
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...",0
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...",1
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N...",0
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul...",1
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge...","[War novel, Roman à clef]",0


In [80]:
## let's try and vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

X = v.fit_transform(df['summary'])

X


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [81]:
## in theory the above cell vectorized all of our text, let's get the Y values from the df (the yes or no value)

Y = df['Science Fiction']
Y

0        0
1        1
2        0
4        1
5        0
        ..
16549    1
16551    0
16555    0
16556    0
16558    0
Name: Science Fiction, Length: 12841, dtype: int64

In [90]:
## ok, lets try and fit this shit into the algorithm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# split data set into test/train
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42)



svc = SVC()
svc.fit(X_train,y_train)



SVC()

In [95]:
svc.predict(X_test)



array([0, 0, 0, ..., 0, 1, 0])

In [96]:
## let's look at some metrics

from sklearn import metrics

